In [107]:
from pyspark.sql.functions import mean
from pyspark.sql.functions import when
from pyspark.sql import functions as F
import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils

In [108]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types
spark = SparkSession.builder.getOrCreate()

In [109]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [111]:
import os
java8_location = "/Users/shwethareddyreddymalli/Downloads/openlogic-openjdk-8u262-b10-mac-x64/jdk1.8.0_262.jdk/Contents"
os.environ['JAVA_HOME'] = java8_location

### Intial Data cleaning and analysis for better understanding of data

#### Loading the csv file and checking for null values if any

In [113]:
DF = spark.read.csv('weatherAUS.csv',inferSchema =True, header = True)

In [142]:
for i in DF.columns:
        count = DF.filter(DF[i] == 'NA').count()
        print('Null Counts',i,'are:',count)

Null Counts MinTemp are: 1485
Null Counts MaxTemp are: 1261
Null Counts Rainfall are: 3261
Null Counts WindGustDir are: 10326
Null Counts WindGustSpeed are: 10263
Null Counts WindDir9am are: 10566
Null Counts WindDir3pm are: 4228
Null Counts WindSpeed9am are: 1767
Null Counts WindSpeed3pm are: 3062
Null Counts Humidity9am are: 2654
Null Counts Humidity3pm are: 4507
Null Counts Pressure9am are: 15065
Null Counts Pressure3pm are: 15028
Null Counts RainToday are: 3261
Null Counts RainTomorrow are: 3267


#### I have dropped the columns which are independent and also the columns which had null values more than 60%

In [115]:
DF = DF.drop('Date','Location','Evaporation','Sunshine','Cloud9am','Cloud3pm','Temp9am','Temp3pm')

In [144]:
numeric_col = [DF['MinTemp'],DF['MaxTemp'],DF['Rainfall'],DF['WindGustSpeed'],DF['WindSpeed9am'],DF['WindSpeed3pm'],DF['Humidity9am'],DF['Humidity3pm'],DF['Pressure9am']]
    


#### Trying to calculate the average values

In [117]:
for i in numeric_col:
        average = DF.select([mean(i)]).show()
        average

+------------------+
|      avg(MinTemp)|
+------------------+
|12.194034380968981|
+------------------+

+------------------+
|      avg(MaxTemp)|
+------------------+
|23.221348275647017|
+------------------+

+----------------+
|   avg(Rainfall)|
+----------------+
|2.36091814991655|
+----------------+

+------------------+
|avg(WindGustSpeed)|
+------------------+
| 40.03523007167319|
+------------------+

+------------------+
| avg(WindSpeed9am)|
+------------------+
|14.043425914971502|
+------------------+

+------------------+
| avg(WindSpeed3pm)|
+------------------+
|18.662656778887342|
+------------------+

+-----------------+
| avg(Humidity9am)|
+-----------------+
|68.88083133761887|
+-----------------+

+----------------+
|avg(Humidity3pm)|
+----------------+
|51.5391158755046|
+----------------+

+------------------+
|  avg(Pressure9am)|
+------------------+
|1017.6499397983008|
+------------------+



Updating the numeric values with the mean

In [118]:
updatedmint  = df.withColumn("MinTemp",F.when(F.col("MinTemp") == 'NA',12.2).otherwise(F.col("MinTemp")))
updatedmaxt  = updatedmint.withColumn("MaxTemp",F.when(F.col("MaxTemp") == 'NA',23.2).otherwise(F.col("MaxTemp")))
updatedrainf = updatedmaxt.withColumn("Rainfall",F.when(F.col("Rainfall") == 'NA',2.3).otherwise(F.col("Rainfall")))
updatedwind  = updatedrainf.withColumn("WindGustSpeed",F.when(F.col("WindGustSpeed") == 'NA',40.0).otherwise(F.col("WindGustSpeed")))
updatedwinds9= updatedwind.withColumn("WindSpeed9am",F.when(F.col("WindSpeed9am") == 'NA',14.0).otherwise(F.col("WindSpeed9am")))
updatedwinds3= updatedwinds9.withColumn("WindSpeed3pm",F.when(F.col("WindSpeed3pm") == 'NA',18.6).otherwise(F.col("WindSpeed3pm")))
updatedhum9  = updatedwinds3.withColumn("Humidity9am",F.when(F.col("Humidity9am") == 'NA',68.8).otherwise(F.col("Humidity9am")))
updatedhum3  = updatedhum9.withColumn("Humidity3pm",F.when(F.col("Humidity3pm") == 'NA',51.5).otherwise(F.col("Humidity3pm")))
updatedpres9 = updatedhum3.withColumn("Pressure9am",F.when(F.col("Pressure9am") == 'NA',1017.7).otherwise(F.col("Pressure9am")))
inputted_numeric = updatedpres9.withColumn("Pressure3pm",F.when(F.col("Pressure3pm") == 'NA',1015.3).otherwise(F.col("Pressure3pm")))

In [119]:
updatedwindgust = inputted_numeric.withColumn("WindGustDir",F.when(F.col("WindGustDir") == 'NA','W').otherwise(F.col("WindGustDir")))
updatedwindir9 = updatedwindgust.withColumn("WindDir9am",F.when(F.col("WindDir9am") == 'NA','N').otherwise(F.col("WindDir9am")))
updatedwindir3 = updatedwindir9.withColumn("WindDir3pm",F.when(F.col("WindDir3pm") == 'NA','SE').otherwise(F.col("WindDir3pm")))
updatedraint = updatedwindir3.withColumn("RainToday",F.when(F.col("RainToday") == 'NA','No').otherwise(F.col("RainToday")))
inputted_weather = updatedraint.withColumn("RainTomorrow",F.when(F.col("RainTomorrow") == 'NA','No').otherwise(F.col("RainTomorrow")))

In [120]:
for i in inputted_weather.columns:
        count = inputted_weather.filter(inputted_weather[i] == 'NA').count()
        print('Total null counts for',i,'are:',count)

Total null counts for MinTemp are: 0
Total null counts for MaxTemp are: 0
Total null counts for Rainfall are: 0
Total null counts for WindGustDir are: 0
Total null counts for WindGustSpeed are: 0
Total null counts for WindDir9am are: 0
Total null counts for WindDir3pm are: 0
Total null counts for WindSpeed9am are: 0
Total null counts for WindSpeed3pm are: 0
Total null counts for Humidity9am are: 0
Total null counts for Humidity3pm are: 0
Total null counts for Pressure9am are: 0
Total null counts for Pressure3pm are: 0
Total null counts for RainToday are: 0
Total null counts for RainTomorrow are: 0


In [121]:
numeric_col = ['MinTemp','MaxTemp','Rainfall','WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Humidity3pm','Humidity9am','Pressure3pm']

In [122]:
for i in numeric_col:
            inputted_weather = inputted_weather.withColumn(i, inputted_weather[i].cast(DoubleType()))

#### StringIndexer and Onehotencoder implementation

In [123]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from distutils.version import LooseVersion

In [124]:
categoricalColumns = ["WindGustDir","WindDir9am","WindDir3pm","RainToday","RainTomorrow"]
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")                    
    if LooseVersion(pyspark.__version__) < LooseVersion("3.0"):
        from pyspark.ml.feature import OneHotEncoderEstimator
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    else:
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

#### Joining the columns using Vector Assembler

In [125]:
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numeric_col
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [126]:
stages

[StringIndexer_8342181a26f6,
 OneHotEncoder_7540f5c7ac3a,
 StringIndexer_96a67864a065,
 OneHotEncoder_60ff1f6f35c2,
 StringIndexer_a16a0db4b6eb,
 OneHotEncoder_8b0a429ede61,
 StringIndexer_487053cfba90,
 OneHotEncoder_33e4ff2ff139,
 StringIndexer_dc6f3df4776a,
 OneHotEncoder_fd6c24afea43,
 VectorAssembler_3eef35fef2ce]

In [127]:
dt = DecisionTreeClassifier(labelCol='RainTomorrowIndex' , featuresCol="features")

### Performing decision tree classifier for further usage

#### Listing all the parameters

In [128]:
dtparamGrid = (ParamGridBuilder()
              .addGrid(dt.impurity, ["gini", "entropy"])
            .addGrid(dt.maxDepth, [5])
            .addGrid(dt.maxBins, [5, 10, 15])
            .addGrid(dt.minInfoGain, [0.0, 0.2, 0.4])
            .build())

In [129]:
dtevaluator = BinaryClassificationEvaluator().setLabelCol("RainTomorrowIndex")

#### Defining the evaluator and number of folds for validation

In [130]:
dtcv = CrossValidator(estimator = dt,
                      estimatorParamMaps = dtparamGrid,
                      evaluator = dtevaluator,
                      numFolds = 4)

In [131]:
stages += [dtcv]

#### The data will be splitted 

In [132]:
(trainingData, testData) = inputted_weather.randomSplit([0.8, 0.2], seed=12345)

#### Stages are assigned and then the data is processed

In [133]:
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(trainingData)
preppedDataDF = pipelineModel.transform(testData)

Exception ignored in: <function JavaModelWrapper.__del__ at 0x7ff08a3630d0>     
Traceback (most recent call last):
  File "/Users/shwethareddyreddymalli/opt/anaconda3/lib/python3.9/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'BinaryClassificationMetrics' object has no attribute '_sc'
Exception ignored in: <function JavaModelWrapper.__del__ at 0x7ff08a3630d0>
Traceback (most recent call last):
  File "/Users/shwethareddyreddymalli/opt/anaconda3/lib/python3.9/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'BinaryClassificationMetrics' object has no attribute '_sc'
Exception ignored in: <function JavaModelWrapper.__del__ at 0x7ff08a3630d0>
Traceback (most recent call last):
  File "/Users/shwethareddyreddymalli/opt/anaconda3/lib/python3.9/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._

In [134]:
model = pipelineModel.stages[-1].bestModel

In [135]:
model.extractParamMap()

{Param(parent='DecisionTreeClassifier_afb104a26bbe', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='DecisionTreeClassifier_afb104a26bbe', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='DecisionTreeClassifier_afb104a26bbe', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='DecisionTreeClassifier_afb104a26bbe', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gin

In [136]:
preds = preppedDataDF.select('RainTomorrowIndex','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['RainTomorrowIndex'])))

In [145]:
metrics = BinaryClassificationMetrics(preds)

22/04/27 23:51:44 ERROR Executor: Exception in task 0.0 in stage 5608.0 (TID 12318)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/shwethareddyreddymalli/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 481, in main
    raise RuntimeError(("Python in worker has different version %s than that in " +
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:555)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:713)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:695)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:508)
	at 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5608.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5608.0 (TID 12318) (shwethas-mbp executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/shwethareddyreddymalli/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 481, in main
    raise RuntimeError(("Python in worker has different version %s than that in " +
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:555)
	at org.apache.spark.api.python.PythonRunner$$anon

I am unable to trigger this runtime error. This says the python version in driver than that of worker. Tried multiple options but didnt work for me.

In [147]:
preppedDataDF.show(5)

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+----------------+-------------------+---------------+------------------+---------------+------------------+--------------+-----------------+-----------------+--------------------+--------------------+-------------+-----------+----------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|WindGustDirIndex|WindGustDirclassVec|WindDir9amIndex|WindDir9amclassVec|WindDir3pmIndex|WindDir3pmclassVec|RainTodayIndex|RainTodayclassVec|RainTomorrowIndex|RainTomorrowclassVec|            features|rawPrediction|probability|prediction|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+-----------